# Stock Performance

See the gains/losses of your stocks in certain period (e.g. last 1 month, 3 months, 1 year etc.) or a custom date range

#### Enter the list of stock tickers you would like to analyze

In [25]:
crazy_stocks = ['AAPL', 'AMZN', 'BABA', 'BIGC', 'ETSY', 'FB', 'GOOGL', 'JD', 'LYFT', 'LI',
                'MSFT', 'NFLX', 'NIO', 'NVDA', 'PINS', 'PYPL', 'ROKU', 'SHOP', 'SNAP', 'SPLK',
                'SQ', 'TSLA', 'UBER', 'W', 'WMT', 'XPEV', 'ZM', 
                'QQQ', 'FNGU', 'VFINX', 'VGT', 'VTSMX']

In [26]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime, timezone
from datetime import timedelta

from yahoo_earnings_calendar import YahooEarningsCalendar
import yfinance as yf

In [27]:
""" Download the stock prices using yfinance API """

tickers = crazy_stocks

tomorrow = '{}'.format(date.today() + timedelta(days=1))
y1_ago = '{}'.format(date.today() - timedelta(days = 366))

tickers_str = ' '.join(tickers)

data = yf.download(tickers_str, y1_ago, tomorrow, group_by='ticker')


print("Closing Prices")
c_prices = data.iloc[-1].loc[(slice(None), 'Close')].sort_index()
print(c_prices.round(2))

[*********************100%***********************]  32 of 32 completed
Closing Prices
AAPL      111.20
AMZN     3162.78
BABA      307.94
BIGC       79.65
ETSY      139.64
FB        267.67
FNGU      191.38
GOOGL    1510.80
JD         80.08
LI         18.69
LYFT       22.23
MSFT      202.68
NFLX      486.24
NIO        27.51
NVDA      505.08
PINS       49.25
PYPL      192.31
QQQ       271.64
ROKU      217.40
SHOP     1026.71
SNAP       42.86
SPLK      205.72
SQ        167.13
TSLA      406.02
UBER       33.65
VFINX     302.08
VGT       300.62
VTSMX      81.31
W         282.74
WMT       140.04
XPEV       19.02
ZM        516.01
Name: 2020-10-28 00:00:00, dtype: float64


In [28]:
""" Compute last 1m, 3m, 6m and 1year gains """

y1_ago_price = data.iloc[0].loc[(slice(None), 'Close')]
m6_ago_price = data.iloc[-26 * 5].loc[(slice(None), 'Close')]
m3_ago_price = data.iloc[-13 * 5].loc[(slice(None), 'Close')]
m1_ago_price = data.iloc[-4 * 5].loc[(slice(None), 'Close')]
curr_price = data.iloc[-1].loc[(slice(None), 'Close')]


m1_gain = (curr_price - m1_ago_price) / m1_ago_price
m3_gain = (curr_price - m3_ago_price) / m3_ago_price
m6_gain = (curr_price - m6_ago_price) / m6_ago_price
y1_gain = (curr_price - y1_ago_price) / y1_ago_price

curr_date = np.datetime_as_string(data.index.values[-1], unit='D')

df = pd.DataFrame([curr_price, m1_gain, m3_gain, m6_gain, y1_gain],
                  index=["Price on " + curr_date,  '1m', '3m', '6m', '1y'])

df = df.T.sort_values(by='3m', ascending=False)

format_dict = { col_name : '{:.2%}' for col_name in ['1m', '3m', '6m', '1y', 'Gain']}
df.style.format(format_dict)

,Price on 2020-10-28,1m,3m,6m,1y
NIO,27.51,26.42%,116.61%,749.07%,1758.78%
ZM,516.01,6.84%,104.45%,213.49%,675.26%
PINS,49.25,10.60%,97.39%,137.01%,87.62%
SNAP,42.86,60.04%,87.16%,153.46%,191.96%
FNGU,191.38,-5.56%,48.16%,210.38%,366.33%
ROKU,217.4,8.52%,43.43%,69.01%,48.01%
TSLA,406.02,-9.40%,35.42%,154.16%,519.48%
SQ,167.13,-0.81%,30.01%,162.99%,166.13%
ETSY,139.64,6.04%,28.48%,101.56%,143.61%
JD,80.08,1.92%,27.94%,75.69%,152.38%


In [30]:
""" Sort the stocks by 1 year gains"""

df = df.sort_values(by='1y', ascending=False)
df.style.format(format_dict)

,Price on 2020-10-28,1m,3m,6m,1y
NIO,27.51,26.42%,116.61%,749.07%,1758.78%
ZM,516.01,6.84%,104.45%,213.49%,675.26%
TSLA,406.02,-9.40%,35.42%,154.16%,519.48%
FNGU,191.38,-5.56%,48.16%,210.38%,366.33%
SHOP,1026.71,-2.63%,-2.55%,63.34%,215.98%
SNAP,42.86,60.04%,87.16%,153.46%,191.96%
SQ,167.13,-0.81%,30.01%,162.99%,166.13%
W,282.74,-10.91%,16.91%,119.50%,158.66%
JD,80.08,1.92%,27.94%,75.69%,152.38%
NVDA,505.08,-7.25%,20.65%,70.01%,144.25%


### This section lets you see the gains in the custom date range
Update the date1 and date2. Note both have to be withing 1 year as the data is fecteched just for a year

In [31]:
date1 = '2020-02-13'
date2 = '2020-03-24'

In [32]:
price1 = data.loc[date1].loc[(slice(None), 'Close')]
price2 = data.loc[date2].loc[(slice(None), 'Close')]
price1.name = date1
price2.name = date2

diff = (price2 - price1) / price1
diff.name = "Gain"


change = pd.concat([price1, price2, diff], axis=1)
change = change.sort_values(by="Gain", ascending=False)


change.style.format(format_dict)

,2020-02-13,2020-03-24,Gain
ZM,89.86,135.18,50.43%
WMT,117.44,115.03,-2.05%
JD,42.01,41.01,-2.38%
NFLX,381.4,357.32,-6.31%
NVDA,270.78,249.18,-7.98%
AMZN,2149.87,1940.1,-9.76%
BABA,220.36,185.75,-15.71%
MSFT,183.71,148.34,-19.25%
PYPL,120.51,97.29,-19.27%
SHOP,532.97,430,-19.32%


In [23]:
yec = YahooEarningsCalendar()

edates = {}
for s in crazy_stocks:
    try:
        ed = yec.get_next_earnings_date(s)
        dt = datetime.fromtimestamp(ed, timezone.utc)
        if dt < datetime.now(timezone.utc):
            print('The earning date is past for ' + s)
        edates[s] = dt
    except:
        print("No earnings date for ", s)
        pass
    
print("Earning dates found for {} stocks".format(len(edates)))

The earning date is past for AAPL
The earning date is past for AMZN
The earning date is past for ETSY
The earning date is past for FB
The earning date is past for GOOGL
The earning date is past for JD
No earnings date for  LI
The earning date is past for NIO
The earning date is past for PINS
The earning date is past for SHOP
The earning date is past for SNAP
No earnings date for  VGT
No earnings date for  QQQ
No earnings date for  FNGU
No earnings date for  XPEV
Earning dates found for 25 stocks


In [24]:
edates_sorted = sorted(edates.items(), key=lambda x: x[1])
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")

for e in edates_sorted:
    dt = e[1]
    d = dt.strftime('%Y-%m-%d')
    wd = dt.date().weekday()
    print('{}\t{}\t{}'.format(e[0], d, weekDays[wd]))

NIO	2020-05-28	Thursday
JD	2020-08-11	Tuesday
SNAP	2020-10-20	Tuesday
ETSY	2020-10-28	Wednesday
PINS	2020-10-28	Wednesday
AAPL	2020-10-29	Thursday
AMZN	2020-10-29	Thursday
FB	2020-10-29	Thursday
GOOGL	2020-10-29	Thursday
SHOP	2020-10-29	Thursday
PYPL	2020-11-02	Monday
W	2020-11-03	Tuesday
BABA	2020-11-05	Thursday
BIGC	2020-11-05	Thursday
ROKU	2020-11-05	Thursday
SQ	2020-11-05	Thursday
UBER	2020-11-05	Thursday
LYFT	2020-11-10	Tuesday
WMT	2020-11-17	Tuesday
NVDA	2020-11-18	Wednesday
SPLK	2020-11-19	Thursday
ZM	2020-12-04	Friday
NFLX	2021-01-19	Tuesday
MSFT	2021-01-27	Wednesday
TSLA	2021-01-27	Wednesday
